# Análisis Binario — Clasificación de Malaria
Proyecto de Inteligencia Artificial (Ingeniería Biomédica)

_Generado: 2025-10-25_


In [ ]:
# === Librerías ===
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
print('✅ Librerías importadas')


In [ ]:
# === Cargar dataset (train.csv en la misma carpeta) ===
df = pd.read_csv('train.csv')
print('✅ Dataset cargado con', len(df), 'registros')
df.head()


In [ ]:
# === Separar variables y split ===
X = df.drop(columns=['label'])
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape


In [ ]:
# === Función de métricas ===
def calcular_metricas(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec  = recall_score(y_true, y_pred)  # sensibilidad
    spec = tn / (tn + fp)
    f1   = f1_score(y_true, y_pred)
    return acc, prec, rec, spec, f1, cm
print('✅ Función de métricas lista')


In [ ]:
# === Modelos a comparar ===
modelos = {
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=1000))
    ]),
    'K-Nearest Neighbors': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', KNeighborsClassifier(n_neighbors=5))
    ]),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42),
    'SVM': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', SVC(kernel='rbf', C=1.0))
    ])
}
print('✅ Modelos definidos')


In [ ]:
# === Entrenamiento y tabla de métricas ===
res = []
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    acc, prec, rec, spec, f1, cm = calcular_metricas(y_test, y_pred)
    res.append([nombre, acc, prec, rec, spec, f1])

tabla = pd.DataFrame(res, columns=['Modelo','Accuracy','Precision','Recall (Sensibilidad)','Specificity','F1-Score'])
tabla.sort_values(by='F1-Score', ascending=False).reset_index(drop=True)


### 🔍 Mejor modelo y justificación
El modelo seleccionado fue **Random Forest**, por su mejor equilibrio entre **Sensibilidad**, **Precisión** y **F1-Score**. En contexto biomédico priorizamos la **sensibilidad** para no omitir células infectadas.


### ⚠️ Análisis de error
- Falsos negativos: células infectadas muy similares a sanas o con baja carga parasitaria.
- Falsos positivos: artefactos de tinción/iluminación que confunden al modelo.
- Mejoras: aumento de datos (data augmentation), balanceo de clases y ajuste de hiperparámetros.
